In [1]:
import weather_bot
import twitter_bot
import location_bot
import news_bot
from datetime import datetime, timedelta

In [2]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

In [3]:
current_weather = weather_bot.current_weather()
detail_weather = weather_bot.detail_weather_forecast()
tomorrow_weather = detail_weather['daily'][1]
next_hour_weather = detail_weather['hourly'][1]
local_time = datetime.now()
date_now = local_time.strftime('%Y-%m-%d %A')
time_now = local_time.strftime('%H:%M:%S %p')
schedule = "You have no scheduled activity."
trends, top10_trend_str = twitter_bot.get_trend_in_my_loc()
most_popular_tweet = twitter_bot.get_most_popular_tweet()
twitter_first_tweet = twitter_bot.get_first_tweet_on_timeline()

In [4]:
import yake
def keyword_extractor(sentence):
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.1
    numOfKeywords = 10
    custom_kw_extractor = \
    yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                          dedupLim=deduplication_threshold, 
                          top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(sentence)
    keywords.sort(key=lambda x: x[1], reverse=True)
    # return the highest possible keyword
    return keywords[0][0]

In [5]:
from requests import get
from bs4 import BeautifulSoup
def get_google_search(query):
#     query = "quantum physics"
    raw = get(f"https://www.google.com/search?q={query}")
    soup = BeautifulSoup(raw.text, 'html')
    description = soup.find_all("div",{"class":"BNeawe s3v9rd AP7Wnd"})
    # return the text of the first result
    return description[0].text

In [6]:
from googlesearch import search
def get_google_1result(query):
#     query = "samira"
    url_list = [*search(query, tld="co.in", num=5, stop=5, pause=2)]
    # return the first url
    print(url_list[0])

In [7]:
def kelvinToCelsius(kelvin):
    return kelvin - 273.15

def precip_likely(precip):
    if precip < 10:
        return "Precipitation is very unlikely."
    elif (precip >= 10)&(precip < 35):
        return "Precipitation is unlikely."
    elif (precip >= 35)&(precip < 65):
        return "It's likely to have some precipitation."
    elif (precip >= 65)&(precip < 90):
        return "It's very likely to have some precipitation in the next hour."
    else:
        return "It's most likely to have precipitation in the next hour."

In [8]:
current_weather_str = \
f"The weather right now is {current_weather['weather'][0]['description']}"+\
f"\nwith temperature of {kelvinToCelsius(current_weather['main']['temp']):.0f} degree celsius"+\
f"\nand feels like: {kelvinToCelsius(current_weather['main']['feels_like']):.0f} degree celsius."+\
f"\nHumidity is {current_weather['main']['humidity']:.1f} percents"

tomorrow_weather_str = \
f"The weather tomorrow will be {next_hour_weather['weather'][0]['description']}"+\
f"\nwith temperature of {kelvinToCelsius(tomorrow_weather['temp']['day']):.0f} degree celsius during the day"+\
f"\nand {kelvinToCelsius(tomorrow_weather['temp']['night']):.0f} degree celsius during the night."+\
f"\nFeels like {kelvinToCelsius(tomorrow_weather['feels_like']['day']):.0f} degree celsius during the day"+\
f"\nand {kelvinToCelsius(tomorrow_weather['feels_like']['night']):.0f} degree celsius during the night"+\
"\nSunrise will be "+(datetime.utcfromtimestamp(tomorrow_weather['sunrise'])+
              timedelta(hours=-5)).strftime('%H:%M')+" in the morning, "+\
"\nand sunset will be "+(datetime.utcfromtimestamp(tomorrow_weather['sunset'])+
              timedelta(hours=-5)).strftime('%H:%M')+" in the afternoon."

next_hour_weather_str = \
f"The weather will be {next_hour_weather['weather'][0]['description']} in the next hour, "+\
f"\nwith temperature of {kelvinToCelsius(next_hour_weather['temp']):.0f} degree celsius"+\
f"\nand humidity of {next_hour_weather['humidity']:.1f} percents."+\
f"\nUV Index is {next_hour_weather['uvi']:.1f}"+\
"\n"+precip_likely(detail_weather['minutely'][0]['precipitation'])

In [9]:
greet_list = [
    {
        "tag": "Greeting",
        "patterns": ["Hello", "How are you?", "Hi there", "Hi", "Whats up","good morning","good afternoon",
                     "good evening", "How's it going","How's everything","Howdy"],
        "responses": ["Howdy", "Hello", "How are you doing?", "Greetings!", "How do you do?"]
     }]

bye_list = [
    {
        "tag": "Goodbye",
        "patterns": ["bye", "got to go", "see ya", "adios", "see you soon","end conversation","later",
                     "I gotta go","have a good one","have a nice day","enjoy your evening",
                    "enjoy the rest of your day","good night"],
        "responses": ["It was nice speaking to you.", "We'll be in contact soon.", "Speak soon!",
                     "Keep in touch!"]
     }]

search_list = [
    {
        "tag": "GoogleSearch",
        "patterns": ["How does mumbo-jumbo work?", "can you explalin mumbo-jumbo to me?",
                    "can you show me mumbo-jumbo on google?", "who is mumbo-jumbo",
                    "show me how mumbo-jumbo works","what is mumbo-jumbo",
                    "tell me about mumbo-jumbo","tell me how mumbo-jumbo works",
                    "search something on google","something recommendation in somewhere",
                    "show me things to do in somewhere","show me places to go in somewhere",
                    "show me restaurants in somewhere","show me something in somewhere"],
        "responses": [""]
     },{
        "tag": "MapsSearch",
        "patterns": ["what is the opening hour of mumbo-jumbo","where is mumbo-jumbo",
                    "show me something on the map"],
        "responses": [""]
     },{
        "tag": "NewsSearch",
        "patterns": ["what is happening in mumbo-jumbo","news story about mumbo-jumbo",
                    "what happened in somewhere","show me the news in somewhere"],
        "responses": [""]
     }]

weather_list = [
    {
        "tag": "Current",
        "patterns": [ "what's the weather like today","what's the weather outside","what's the current weather",
                    "how's the weather","how's the weather like","is it cold outside","is it hot today"],
        "responses": [current_weather_str]
    },{
        "tag": "Tomorrow",
        "patterns": [ "what's the weather like tomorrow","how's the weather tomorrow",
                     "what's the weather like the coming day"],
        "responses": [tomorrow_weather_str]
    },{
        "tag": "Next Hour",
        "patterns": [ "what's the weather like in the next hour","is it going to rain",
                     "what's the weather like in the coming hour"],
        "responses": [next_hour_weather_str]
    }]

datetime_list = [
    {
        "tag": "Date",
        "patterns": [ "what day is it today","what's the date today"],
        "responses": ["Today is "+date_now]
    },{
        "tag": "Time",
        "patterns": [ "what time is it","what time is it now"],
        "responses": ["The time is now "+time_now]
    },{
        "tag": "Calendar",
        "patterns": [ "what's on my calendar","what is my schedule","what's on my schedule"],
        "responses": [schedule]
    }]

ai_list = [
    {
        "tag": "Name",
        "patterns": ["who are you","what's your name","what should I call you","introduce yourself",
                     "tell me about yourself"],
        "responses": ["My name is Lulu, your personal debug companion:)",
                     "I'm Lulu, I'm here to solve your debug problems."]
    },{
        "tag": "Age",
        "patterns": [ "how old are you","when are you born","when are you made"],
        "responses": ["I'm 1-day-old, I just born yesterday."]
    }]

twitter_list = [
    {
        "tag": "Trending",
        "patterns": [ "what's trending right now","what's popular on twitter",
                     "what's the hottest topic","show me the trending topics on twitter",
                     "what's on twitter"],
        "responses": [top10_trend_str]
    },{
        "tag": "First Tweet",
        "patterns": [ "what's the first tweet on my timeline","what's on my timeline",
                    "my home timeline"],
        "responses": [twitter_first_tweet]
    },{
        "tag": "Most Popular Tweet",
        "patterns": [ "what's the most popular tweet","what's the trending tweet"],
        "responses": [most_popular_tweet]
    }]

jokes_list = [
    {
        "tag": "General",
        "patterns": [ "tell me a joke","tell me something funny"],
        "responses": ["What do you call a fish wearing a bowtie? Sofishticated.",
                     "Dear Math, grow up and solve your own problems.",
                     "I only know 25 letters of the alphabet. I don't know Y.",
                     "Why does Teemo live in a small house? He doesn't need mushroom.",
                     "Why did the manaless Syndra run from the teamfight? She didn’t have the balls.",
                     "How does Janna shield her allies? With Ease.",
                     "Whats vayne's favorite website? tumblr."]
    }]

In [10]:
data = {
    "Greeting":greet_list,
    "Goodbye":bye_list,
    "Search":search_list,
    "WeatherInfo":weather_list,
    "TwitterInfo":twitter_list,
    "AIInfo":ai_list,
    "DateTimeInfo":datetime_list,
    "Jokes":jokes_list
}

#### Bert-based model testing

In [11]:
# model_name = "bert-base-uncased"
# max_length = 256
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.model_selection import train_test_split
# from transformers import BertTokenizerFast
# tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
# dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers","footers", "quotes"))
# target_names=dataset.target_names
# news_text = dataset.data
# labels = dataset.target
# (train_texts,valid_texts,train_labels,valid_labels)=train_test_split(news_text, labels, test_size=0.3)

# from transformers import BertTokenizerFast, BertForSequenceClassification
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
# model=BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))

# from transformers import Trainer, TrainingArguments
# training_args = TrainingArguments(
#     num_train_epochs=10,
#     per_device_train_batch_size=16,  # batch size per device during training
#     weight_decay=0.01,               # strength of weight decay
#     load_best_model_at_end=True,
#     #logging_steps=200,
#     evaluation_strategy="steps",
#     output_dir="./tmp_trainer/"
# )
# trainer = Trainer(model=model,args=training_args,train_dataset=train_encodings)
# trainer.train()

In [12]:
# from transformers import GPT2Tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [13]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes_outer = []
classes_inner = []
x_patterns = []
y_inner_intents = []
y_outer_intents = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for outer_intent in data.keys():
    for inner_intent in data[outer_intent]:
        for pattern in inner_intent["patterns"]:
            tokens = nltk.word_tokenize(pattern)
            words.extend(tokens)
            x_patterns.append(pattern)
            y_inner_intents.append(inner_intent["tag"])
            y_outer_intents.append(outer_intent)
        
        if inner_intent["tag"] not in classes_inner:
            classes_inner.append(inner_intent["tag"])
    # add the tag to the classes if it's not there already 
    if outer_intent not in classes_outer:
        classes_outer.append(outer_intent)
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes_outer = sorted(set(classes_outer))
classes_inner = sorted(set(classes_inner))

#### Inner Model

In [14]:
# list for training data
training = []
out_empty = [0] * len(classes_inner)
# creating the bag of words model
for idx, doc in enumerate(x_patterns):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated to
    output_row = list(out_empty)
    output_row[classes_inner.index(y_inner_intents[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [15]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model_inner = Sequential()
model_inner.add(Dense(256, input_shape=input_shape, activation="relu"))
model_inner.add(Dropout(0.3))
model_inner.add(Dense(64, activation="relu"))
model_inner.add(Dropout(0.3))
model_inner.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model_inner.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model_inner.summary())
model_inner.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               28672     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                1105      
Total params: 46,225
Trainable params: 46,225
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
3/3 [==============================] - 0s 665us/step - loss: 2.7702 - accuracy: 0.1412
Epoch 2/200
3/

3/3 [==============================] - 0s 665us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 73/200
3/3 [==============================] - 0s 665us/step - loss: 0.0181 - accuracy: 0.9882
Epoch 74/200
3/3 [==============================] - 0s 665us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 75/200
3/3 [==============================] - 0s 665us/step - loss: 0.0045 - accuracy: 1.0000
Epoch 76/200
3/3 [==============================] - 0s 665us/step - loss: 0.0084 - accuracy: 1.0000
Epoch 77/200
3/3 [==============================] - 0s 665us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 78/200
3/3 [==============================] - 0s 665us/step - loss: 0.0050 - accuracy: 1.0000
Epoch 79/200
3/3 [==============================] - 0s 665us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 80/200
3/3 [==============================] - 0s 665us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 81/200
3/3 [==============================] - 0s 332us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 82/200


3/3 [==============================] - 0s 664us/step - loss: 0.1222 - accuracy: 0.9765
Epoch 153/200
3/3 [==============================] - 0s 665us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 154/200
3/3 [==============================] - 0s 665us/step - loss: 0.0197 - accuracy: 0.9882
Epoch 155/200
3/3 [==============================] - 0s 665us/step - loss: 0.0041 - accuracy: 1.0000
Epoch 156/200
3/3 [==============================] - 0s 665us/step - loss: 0.0158 - accuracy: 1.0000
Epoch 157/200
3/3 [==============================] - 0s 333us/step - loss: 0.0137 - accuracy: 0.9882
Epoch 158/200
3/3 [==============================] - 0s 664us/step - loss: 6.4544e-04 - accuracy: 1.0000
Epoch 159/200
3/3 [==============================] - 0s 333us/step - loss: 0.0057 - accuracy: 1.0000
Epoch 160/200
3/3 [==============================] - 0s 664us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 161/200
3/3 [==============================] - 0s 665us/step - loss: 0.0303 - accuracy: 0.9765


#### Outer Model

In [16]:
# list for training data
training = []
out_empty = [0] * len(classes_outer)
# creating the bag of words model
for idx, doc in enumerate(x_patterns):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated to
    output_row = list(out_empty)
    output_row[classes_outer.index(y_outer_intents[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [17]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model_outer = Sequential()
model_outer.add(Dense(256, input_shape=input_shape, activation="relu"))
model_outer.add(Dropout(0.3))
model_outer.add(Dense(64, activation="relu"))
model_outer.add(Dropout(0.3))
model_outer.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model_outer.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model_outer.summary())
model_outer.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               28672     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 520       
Total params: 45,640
Trainable params: 45,640
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
3/3 [==============================] - 0s 665us/step - loss: 1.9477 - accuracy: 0.2353
Epoch 2/200


3/3 [==============================] - 0s 332us/step - loss: 3.5982e-04 - accuracy: 1.0000
Epoch 73/200
3/3 [==============================] - 0s 665us/step - loss: 4.1766e-04 - accuracy: 1.0000
Epoch 74/200
3/3 [==============================] - 0s 665us/step - loss: 8.3621e-04 - accuracy: 1.0000
Epoch 75/200
3/3 [==============================] - 0s 333us/step - loss: 3.7337e-05 - accuracy: 1.0000
Epoch 76/200
3/3 [==============================] - 0s 665us/step - loss: 3.4611e-04 - accuracy: 1.0000
Epoch 77/200
3/3 [==============================] - 0s 332us/step - loss: 1.1956e-04 - accuracy: 1.0000
Epoch 78/200
3/3 [==============================] - 0s 665us/step - loss: 1.8588e-04 - accuracy: 1.0000
Epoch 79/200
3/3 [==============================] - 0s 665us/step - loss: 4.6030e-05 - accuracy: 1.0000
Epoch 80/200
3/3 [==============================] - 0s 665us/step - loss: 1.9680e-04 - accuracy: 1.0000
Epoch 81/200
3/3 [==============================] - 0s 333us/step - loss: 1.9

3/3 [==============================] - 0s 333us/step - loss: 2.8139e-04 - accuracy: 1.0000
Epoch 151/200
3/3 [==============================] - 0s 665us/step - loss: 2.9938e-05 - accuracy: 1.0000
Epoch 152/200
3/3 [==============================] - 0s 665us/step - loss: 2.0568e-05 - accuracy: 1.0000
Epoch 153/200
3/3 [==============================] - 0s 665us/step - loss: 1.4892e-05 - accuracy: 1.0000
Epoch 154/200
3/3 [==============================] - 0s 665us/step - loss: 7.1704e-06 - accuracy: 1.0000
Epoch 155/200
3/3 [==============================] - 0s 332us/step - loss: 1.7712e-05 - accuracy: 1.0000
Epoch 156/200
3/3 [==============================] - 0s 665us/step - loss: 6.3020e-04 - accuracy: 1.0000
Epoch 157/200
3/3 [==============================] - 0s 665us/step - loss: 8.3258e-06 - accuracy: 1.0000
Epoch 158/200
3/3 [==============================] - 0s 665us/step - loss: 1.7163e-04 - accuracy: 1.0000
Epoch 159/200
3/3 [==============================] - 0s 333us/step - 

In [18]:
def clean_text(text): 
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_inner(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    result = model_inner.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def pred_outer(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    result = model_outer.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

In [19]:
def get_response(inner_intent, outer_intent, intents_json): 
    response = ""
    inner_intent_list = intents_json[outer_intent]
    for dictionary in inner_intent_list:
        if dictionary['tag'] == inner_intent:
            response = random.choice(dictionary["responses"])
            break
    return response

In [20]:
from IPython.core.display import display
def lulu_speak():
    display(IPython.display.Audio("res.mp3", rate = 44100, autoplay=True))

In [21]:
# message = input("")
# inner_intent = pred_inner(message, words, classes_inner)[0]
# outer_intent = pred_outer(message, words, classes_outer)[0]
# chat = get_response(inner_intent, outer_intent, data)
# print(inner_intent,outer_intent)

In [22]:
# for speech-to-text
import speech_recognition as sr
# for text-to-speech
from gtts import gTTS
# for language model
import transformers
# for playing audio
import IPython
# for data
import os
import time
import numpy as np
# Building the AI
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
    
    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            print("Listening...")
            audio = recognizer.listen(mic)
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
            
        except:
            print("Me  -->  ERROR")
        
    @staticmethod
    def text_to_speech(text):
        print("Lulu --> ", text)
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        statbuf = os.stat("res.mp3")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
#         os.system('start res.mp3')  #if you are using mac->afplay or else for windows->start
#         os.system("play "+"res.mp3"+" tempo 1.5")
#         os.system("close res.mp3")
        lulu_speak()
        time.sleep(int(50*duration))
        os.remove("res.mp3")
    
    def wake_up(self, text):
        return True if self.name in text.lower() else False
   
    @staticmethod
    def logging(self, lulu_text, inner_intent, outer_intent):
        log_list = [self.text, lulu_text, inner_intent, outer_intent, datetime.now().strftime('%H:%M:%S %p')]
        return log_list

In [29]:
# Running the AI
if __name__ == "__main__":
    ai = ChatBot(name="Lulu the Piggy")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    inner_intent, outer_intent = "", ""
    ex = True
    while ex:
        ai.speech_to_text()
        ## wake up
        if ai.wake_up(ai.text) is True:
            answer = "Hello I am Lulu the Piggy, what can I do for you?"
        ## respond politely
        elif any(i in ai.text for i in ["thank","thanks","appreciate"]):
            answer = np.random.choice(["You're welcome!","Anytime!","No problem!","Cool!","I'm here if you need me!"])
        elif any(i in ai.text for i in ["exit","bye","close"]):
            answer = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","Peace out!"])
            ex = False
        ## conversation
        else:   
            if ai.text == "ERROR":
                answer = "Sorry, come again?"
            else:
                ## use the 2 deep neural nets
                inner_intent = pred_inner(ai.text, words, classes_inner)[0]
                outer_intent = pred_outer(ai.text, words, classes_outer)[0]
                if inner_intent == "GoogleSearch":
                    get_google_1result(ai.text)
                    chat = get_google_search(ai.text)
                    answer = str(chat)
                elif inner_intent == "MapsSearch":
                    query = keyword_extractor(ai.text)
                    location = location_bot.get_1st_location(query)
                    html_text = location['photos'][0]['html_attributions'][0]
                    soup = BeautifulSoup(html_text, 'html.parser')
                    print("Photo link: "+soup.find('a').get('href'))
                    if 'establishment' in location['types']:
                        answer = "Name: "+location['name']
                        answer += "\nAddress: "+location['formatted_address']
                        answer += "\nOpen now: "+("yes." if location['opening_hours']['open_now'] else "no.")
                        answer += "\nRating: "+str(location['rating'])
                        try:
                            answer += "\nPrice level: "+"$"*int(location['price_level'])
                        except:
                            pass
                    else:
                        answer = "Not an establishment, unable to print results."
                elif inner_intent == "NewsSearch":
                    query = keyword_extractor(ai.text)
                    news = news_bot.get_1st_news(query)
                    try:
                        print("Link: "+news['link'])
                        answer = "Title: "+news['title']
                    except:
                        answer = "No news available at this time."
                else:
                    chat = get_response(inner_intent, outer_intent, data)
                    answer = str(chat)
        ## convert answer to speech
        if len(answer) > 0:
            ai.text_to_speech(answer)
        else:
            answer = "Sorry, I don't understand you. Could you say again?"
            ai.text_to_speech(answer)
        if outer_intent == "Goodbye":
            ex = False
    print("----- Closing down "+ai.name+" -----")

----- Starting up Lulu the Piggy -----
Listening...
Me  -->  where is McDonald
Photo link: https://maps.google.com/maps/contrib/115004011304798854894
Lulu -->  Name: McDonald's
Address: 325 Grove St, Jersey City, NJ 07303, United States
Open now: yes.
Rating: 3.6
Price level: $


Listening...
Me  -->  show me bubble tea on the map
Photo link: https://maps.google.com/maps/contrib/112463972621193838876
Lulu -->  Name: Fuxion prunex eeuu
Address: Wall St, New York, NY 10005, United States
Open now: yes.
Rating: 5


Listening...
Me  -->  talk about your telepathic
Lulu -->  My name is Lulu, your personal debug companion:)


Listening...
Me  -->  goodbye
Lulu -->  Tata


----- Closing down Lulu the Piggy -----
